# Samplers

Samplers let you train models on images in special ways. That sounds a bit abstract, but it can be clearly stated that you will use samplers whenever you are training on anything other than the entire image. The most common uses of samplers are for training on slices, patches, or blocks of full images.

Training on parts of images is actually quite common in medical imaging AI tasks, primarily because 3D medical images are quite large in size and usually only contain a lot of irrelevant space in the image. The process of correctly and efficiently sampling these parts of images is often a struggle to do manually, which is why nitrain aims to make this process simple!

Samplers are used with loaders (presented in the next tutorial), but they are applied to datasets. And since they can actually be used on their own, it is best to present them here.

Let's start by creating a nitrain dataset:

In [ ]:
import ants
import numpy as np
from nitrain import datasets, loaders, samplers, transforms as tx

dataset = datasets.FolderDataset(base_dir='/Users/ni5875cu/Desktop/ds004711',
                                 x={'pattern': 'sub-*/anat/*_T1w.nii.gz', 'exclude': '**run-02*'},
                                 y={'file': 'participants.tsv', 'column': 'age'},
                                 x_transforms=[tx.Resample((4,4,4), use_spacing=True)])
dataset.x = dataset.x[:7]
dataset.y = dataset.y[:7]

x_raw, y_raw = dataset[:3]


## Slice sampler

The `SliceSampler` class lets you train on 2D slices of 3D images. You can specify the `axis` that you want to take slices along and the `batch_size` for how many slices you want to batch at once. Note that you pass in a list of image to the sampler and get back a generator. This is what it looks like:

In [ ]:
slice_sampler = samplers.SliceSampler(batch_size=24, axis=0, shuffle=True)
slice_batch = slice_sampler(x_raw, y_raw)

for a, b in slice_batch:
    print(len(a))

Here, the 3 3D images have now been split into lists of sliced (2D) images with length 24. This gives us a total of 192 slices served in batches of 24. Since we passed in 3 images the sampler and specified `shuffle=True`, those batches of 24 will include a mix of slices from all 3 images at once.

## Block sampler

The `BlockSampler` class lets you train on 3D blocks of 3D images. Here, you can specify the block size and the stride (i.e., how many steps to take in each direction before taking the next block). You also specify the batch size as before.

In [ ]:
block_sampler = samplers.BlockSampler(block_size=(32,32,24), stride=(6,6,4), batch_size=24)
block_batch = block_sampler(x_raw, y_raw)
a, b = next(iter(block_batch))

From the block sampler, we get blocks of size (32, 32, 24) that are combined into batches of 24. Again, blocks will be shuffled across the 3 images that we passed into the block sampler.

## Patch sampler

Finally, you may want to train on 2D patches of 2D images. The `PatchSampler` class is perfect for that. Let's first create a dataset with 2D images to see how this works.

In [ ]:
x = [ants.image_read(ants.get_data('r16')) for _ in range(10)]
y = list(range(10))
dataset = datasets.MemoryDataset(x, y)
x_raw, y_raw = dataset[:3]

Now that we have a dataset with 3 2D images taken from it, we can create the patch sampler. The patch sampler works nearly identically to the block sampler, except for having on two dimensions.

In [ ]:
patch_sampler = samplers.PatchSampler(patch_size=(64,64), stride=(8,8), batch_size=24)
patch_batch = patch_sampler(x_raw, y_raw)

patch_iter = next(iter(patch_batch))

A cool feature of using `ants` as the medical imaging library is that we can actually put the patches back into the original image using the `decrop_image` function:

In [ ]:
x_raw[0].plot(overlay=ants.decrop_image(a[10], x_raw[0]*0), axis=2)

We will see how useful this is when we use the `explainers` to understand which parts of the image are most important for a trained model.

## Summary

Samplers are quite simple but powerful tools for training on parts of medical images. In the next tutorial, you will see how well they combine with `loaders` to create really efficient and well-organized batches of images that are ready for model training.